# Temporal Analysis: 50 Years of Livestock Farming in Nepal (1972-2021)

## Synthetic Historical Data Generation and Trend Analysis

**Author:** ML Assignment Project  
**Date:** December 2025

---

### Overview

This notebook extends the livestock farming vulnerability analysis by:
1. **Generating synthetic historical data** for 50 years (1972-2021) based on 2021 baseline
2. **Analyzing temporal trends** in agricultural practices across Nepal's districts
3. **Identifying structural changes** in farming systems over time
4. **Clustering evolution** - how farming profiles have changed over decades

## 1. Setup and Library Installation

### 1.1 Install Required Libraries (for Google Colab)


In [ ]:
# Install required libraries (for Google Colab)
%pip install geopandas folium mapclassify -q


### 1.2 Import Libraries


In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    silhouette_score
)
from sklearn.linear_model import LinearRegression

# Geographic visualization (optional - will handle gracefully if not available)
try:
    import geopandas as gpd
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("Note: geopandas not available. Map visualizations will be skipped.")
    print("Install with: pip install geopandas")

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ All libraries imported successfully!")


## 2. Load 2021 Baseline Data from Files\n

In [ ]:
# Detect environment and set up data loading
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False

if IN_COLAB:
    print("Running in Google Colab environment")
else:
    print("Running in local environment")

In [ ]:
# Define file paths
# Update these paths according to your setup

if IN_COLAB:
    # Select/upload CSV files in Colab (file picker)
    from google.colab import files
    print("Please select the three CSV files (you can multi-select):")
    print("  - table-1.-number-of-holdings-and-area-by-district-.csv")
    print("  - table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv")
    print("  - table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv")
    uploaded = files.upload()

    def pick_file(keyword, fallback=None):
        for fname in uploaded.keys():
            if keyword.lower() in fname.lower():
                return fname
        if fallback and fallback in uploaded:
            return fallback
        return list(uploaded.keys())[0]

    holdings_area_path = pick_file('holdings-and-area', 'table-1.-number-of-holdings-and-area-by-district-.csv')
    irrigation_path = pick_file('irrigation', 'table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv')
    livestock_path = pick_file('livestocks-and-poultry', 'table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv')

    print(f"Using files -> holdings: {holdings_area_path}, irrigation: {irrigation_path}, livestock: {livestock_path}")
else:
    # Local paths - update as needed
    base_path = '/Users/eklakdangaura/College/ML/Assignment/Datasets/'
    holdings_area_path = base_path + 'table-1.-number-of-holdings-and-area-by-district-.csv'
    irrigation_path = base_path + 'table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv'
    livestock_path = base_path + 'table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv'

print("File paths configured")

In [ ]:
# Load the three datasets from CSV files
print("Loading datasets from files...")

# Dataset 1: Holdings and Area by District
df_land = pd.read_csv(holdings_area_path, sep='\t')
print(f"1. Holdings & Area Dataset: {df_land.shape[0]} rows, {df_land.shape[1]} columns")

# Dataset 2: Irrigation Data
df_irr = pd.read_csv(irrigation_path, sep='\t')
print(f"2. Irrigation Dataset: {df_irr.shape[0]} rows, {df_irr.shape[1]} columns")

# Dataset 3: Livestock Data
df_live = pd.read_csv(livestock_path)
print(f"3. Livestock Dataset: {df_live.shape[0]} rows, {df_live.shape[1]} columns")

print("\n✓ All datasets loaded successfully from files!")

### 2.1 Exploratory Data Analysis (EDA)


In [ ]:
# Explore Holdings & Area Dataset
print("=" * 60)
print("DATASET 1: Holdings and Area by District")
print("=" * 60)
print(f"\nShape: {df_land.shape}")
print(f"\nColumns: {df_land.columns.tolist()}")
print("\nData Types:")
print(df_land.dtypes)
print("\nFirst 5 rows:")
display(df_land.head())


In [ ]:
# Explore Irrigation Dataset
print("=" * 60)
print("DATASET 2: Irrigation Data")
print("=" * 60)
print(f"\nShape: {df_irr.shape}")
print(f"\nColumns: {df_irr.columns.tolist()}")
print("\nData Types:")
print(df_irr.dtypes)
print("\nFirst 5 rows:")
display(df_irr.head())


In [ ]:
# Explore Livestock Dataset
print("=" * 60)
print("DATASET 3: Livestock Data")
print("=" * 60)
print(f"\nShape: {df_live.shape}")
print(f"\nColumns: {df_live.columns.tolist()}")
print("\nData Types:")
print(df_live.dtypes)
print("\nFirst 5 rows:")
display(df_live.head())


In [ ]:
# Summary Statistics for all datasets
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

print("\n1. Holdings & Area Dataset Statistics:")
display(df_land.describe())

print("\n2. Irrigation Dataset Statistics:")
display(df_irr.describe())

print("\n3. Livestock Dataset Statistics:")
display(df_live.describe())


### 2.2 Data Preprocessing and District Name Standardization


In [ ]:
# Standardize district names and merge datasets
def standardize_district_name(name):
    if pd.isna(name):
        return name
    return str(name).strip().lower()

# Clean district names
df_land['district_clean'] = df_land['Districts'].apply(standardize_district_name)
df_irr['district_clean'] = df_irr['Districts'].apply(standardize_district_name)
df_live['district_clean'] = df_live['District'].apply(standardize_district_name)

# Select relevant columns
df_land_select = df_land[['district_clean', 'Districts', 'Number of holdings', 
                          'Total wet  area (ha)', 'Total dry  area (ha)', 'Total  area (ha)']].copy()
df_land_select.columns = ['district_clean', 'Districts', 'Number of holdings', 
                          'wet_area_ha', 'dry_area_ha', 'total_area_ha']

df_irr_select = df_irr[['district_clean', 'No. of holdings reporting irrigation', 
                        'Total area (ha) of irrigation']].copy()
df_irr_select.columns = ['district_clean', 'holdings_with_irrigation', 'irrigated_area_ha']

# Handle livestock columns
livestock_cols = ['district_clean', 'Total number of holdings', 'Number of holdings reporting livestock',
                  'No. of  cattles', 'No. of buffalo', 'No. of goat/chyangra', 
                  'No. of pigs/boar', 'No. of poultry(chicken)', 'No. of sheep']
df_live_select = df_live[livestock_cols].copy()
df_live_select.columns = ['district_clean', 'total_holdings_livestock', 'holdings_with_livestock',
                          'num_cattle', 'num_buffalo', 'num_goats', 'num_pigs', 'num_poultry', 'num_sheep']

# Fill missing values
for col in df_live_select.columns:
    if col != 'district_clean':
        df_live_select[col] = pd.to_numeric(df_live_select[col], errors='coerce').fillna(0)

# Merge all datasets
df_merged = df_land_select.merge(df_irr_select, on='district_clean', how='outer').merge(
    df_live_select, on='district_clean', how='outer')

# Fill any remaining missing values
numeric_cols = df_merged.select_dtypes(include=[np.number]).columns
df_merged[numeric_cols] = df_merged[numeric_cols].fillna(0)

print(f"Merged dataset: {df_merged.shape[0]} districts, {df_merged.shape[1]} columns")
print(f"\nColumns: {df_merged.columns.tolist()}")

In [ ]:
# Check for district name mismatches across datasets
holdings_districts = set(df_land['district_clean'].dropna())
irrigation_districts = set(df_irr['district_clean'].dropna())
livestock_districts = set(df_live['district_clean'].dropna())

# Find common districts
common_districts = holdings_districts & irrigation_districts & livestock_districts
print(f"District Name Verification:")
print("=" * 60)
print(f"Holdings dataset: {len(holdings_districts)} districts")
print(f"Irrigation dataset: {len(irrigation_districts)} districts")
print(f"Livestock dataset: {len(livestock_districts)} districts")
print(f"\nCommon districts across all datasets: {len(common_districts)}")

# Check for mismatches
all_districts = holdings_districts | irrigation_districts | livestock_districts
if len(all_districts) != len(common_districts):
    print("\n⚠️ Districts with potential mismatches:")
    only_holdings = holdings_districts - common_districts
    only_irrigation = irrigation_districts - common_districts
    only_livestock = livestock_districts - common_districts
    
    if only_holdings:
        print(f"  Only in Holdings: {only_holdings}")
    if only_irrigation:
        print(f"  Only in Irrigation: {only_irrigation}")
    if only_livestock:
        print(f"  Only in Livestock: {only_livestock}")
else:
    print("\n✓ All districts match across datasets!")


### 2.3 Merge Datasets and Handle Missing Values


In [ ]:
# Feature Engineering for 2021 baseline
df_merged['Avg_Land_Size'] = df_merged['total_area_ha'] / df_merged['Number of holdings']
df_merged['Pct_Irrigated'] = (df_merged['irrigated_area_ha'] / df_merged['total_area_ha']) * 100
df_merged['Cattle_per_HH'] = df_merged['num_cattle'] / df_merged['Number of holdings']
df_merged['Buffalo_per_HH'] = df_merged['num_buffalo'] / df_merged['Number of holdings']
df_merged['Goat_per_HH'] = df_merged['num_goats'] / df_merged['Number of holdings']
df_merged['Pig_per_HH'] = df_merged['num_pigs'] / df_merged['Number of holdings']
df_merged['Poultry_per_HH'] = df_merged['num_poultry'] / df_merged['Number of holdings']

# Handle infinite values
df_merged = df_merged.replace([np.inf, -np.inf], 0)

# Create 2021 baseline
base_cols = ['Districts', 'district_clean', 'Avg_Land_Size', 'Pct_Irrigated', 
             'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Pig_per_HH', 'Poultry_per_HH']

# Filter out rows without district name
df_2021 = df_merged[base_cols].copy()
df_2021 = df_2021.dropna(subset=['Districts'])
df_2021['Year'] = 2021

print(f"2021 baseline dataset prepared: {df_2021.shape[0]} districts")
print(f"\n2021 Baseline Statistics:")
print(df_2021.describe().round(2))

In [ ]:
# Check for missing values after merge
print("Missing Values Analysis:")
print("=" * 60)
missing = df_merged.isnull().sum()
if missing.any():
    print("Columns with missing values:")
    print(missing[missing > 0])
else:
    print("✓ No missing values in merged dataset!")

print(f"\nMerged dataset shape: {df_merged.shape}")
print(f"Number of districts: {df_merged['district_clean'].nunique()}")
print("\nMerged dataset preview:")
display(df_merged.head())


### 2.4 Feature Engineering and Visualization


In [ ]:
# Feature Engineering Visualizations
print("Visualizing Engineered Features for 2021 Baseline...")

# Define engineered features
eng_features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 
                'Goat_per_HH', 'Pig_per_HH', 'Poultry_per_HH']

# 1. Correlation Heatmap
plt.figure(figsize=(12, 8))
correlation_matrix = df_2021[eng_features].corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, linewidths=0.5, square=True)
plt.title('Correlation Matrix of Engineered Features (2021)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('temporal_feature_correlation.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Correlation heatmap saved")


In [ ]:
# 2. Distribution of Key Features (2021 Baseline)
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for idx, feature in enumerate(eng_features):
    sns.histplot(df_2021[feature], kde=True, ax=axes[idx], color='steelblue')
    axes[idx].set_title(feature.replace('_', ' ').title(), fontweight='bold')
    axes[idx].set_xlabel('')

# Turn off extra subplots
for idx in range(len(eng_features), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Distribution of Engineered Features (2021 Baseline)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('temporal_feature_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Feature distributions saved")


In [ ]:
# 3. Pairplot for Key Livestock Features
key_features = ['Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Poultry_per_HH']
fig = sns.pairplot(df_2021[key_features], diag_kind='kde', plot_kws={'alpha': 0.6})
fig.fig.suptitle('Pairwise Relationships: Livestock Features (2021)', y=1.02, fontsize=14, fontweight='bold')
plt.savefig('temporal_pairplot.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Pairplot saved")

print("\n✓ Feature engineering visualizations complete!")


## 3. Generate Synthetic Historical Data (1972-2020)

In [ ]:
np.random.seed(42)
all_years_data = [df_2021]
current_df = df_2021.copy()

print('Generating 50 years of synthetic data...')
print('Trends: Poultry -8%/yr, Land +0.8%/yr, Irrigation -1%/yr, Pigs -2%/yr')

for year in range(2020, 1971, -1):
    new_df = current_df.copy()
    new_df['Year'] = year
    noise = np.random.uniform(0.98, 1.02, size=len(new_df))
    
    new_df['Poultry_per_HH'] = new_df['Poultry_per_HH'] * 0.92 * noise
    new_df['Avg_Land_Size'] = new_df['Avg_Land_Size'] * 1.008 * noise
    new_df['Pct_Irrigated'] = new_df['Pct_Irrigated'] * 0.99 * noise
    new_df['Pig_per_HH'] = new_df['Pig_per_HH'] * 0.98 * noise
    new_df['Goat_per_HH'] = new_df['Goat_per_HH'] * 1.001 * noise
    new_df['Cattle_per_HH'] = new_df['Cattle_per_HH'] * 1.001 * noise
    new_df['Buffalo_per_HH'] = new_df['Buffalo_per_HH'] * 1.001 * noise
    
    cols = new_df.select_dtypes(include=[np.number]).columns
    new_df[cols] = new_df[cols].clip(lower=0)
    
    all_years_data.append(new_df)
    current_df = new_df

df_long = pd.concat(all_years_data, ignore_index=True)
df_long = df_long.sort_values(['Districts', 'Year']).reset_index(drop=True)

print(f'Generated {len(df_long)} rows (50 years x {len(df_2021)} districts)')
df_long.to_csv('nepal_agriculture_50_years.csv', index=False)
print('Exported to nepal_agriculture_50_years.csv')

## 4. National Trend Analysis

In [ ]:
features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Pig_per_HH', 'Poultry_per_HH']
national_trends = df_long.groupby('Year')[features].mean().reset_index()

fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.flatten()

for idx, feature in enumerate(features):
    ax = axes[idx]
    ax.plot(national_trends['Year'], national_trends[feature], linewidth=2, marker='o', markersize=2)
    ax.set_xlabel('Year')
    ax.set_ylabel(feature.replace('_', ' '))
    ax.set_title(feature.replace('_', ' ').title(), fontweight='bold')
    ax.grid(True, alpha=0.3)
    z = np.polyfit(national_trends['Year'], national_trends[feature], 1)
    p = np.poly1d(z)
    ax.plot(national_trends['Year'], p(national_trends['Year']), '--r', alpha=0.7)

for idx in range(len(features), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Nepal Agricultural Trends: 1972-2021', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('national_trends_50years.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 50-year change analysis
year_1972 = national_trends[national_trends['Year'] == 1972].iloc[0]
year_2021 = national_trends[national_trends['Year'] == 2021].iloc[0]

print('50-YEAR CHANGE ANALYSIS (1972 -> 2021)')
print('='*60)
for feature in features:
    val_1972 = year_1972[feature]
    val_2021 = year_2021[feature]
    pct_change = ((val_2021 - val_1972) / (val_1972 + 0.01)) * 100
    direction = '+' if pct_change > 0 else ''
    print(f'{feature}: 1972={val_1972:.2f} -> 2021={val_2021:.2f} ({direction}{pct_change:.1f}%)')

## 5. Era-Based Clustering

In [ ]:
def assign_era(year):
    if year <= 1985:
        return 'Era 1: Traditional (1972-1985)'
    elif year <= 2000:
        return 'Era 2: Early Modern (1986-2000)'
    else:
        return 'Era 3: Commercial (2001-2021)'

df_long['Era'] = df_long['Year'].apply(assign_era)

era_profiles = df_long.groupby(['Districts', 'Era'])[features].mean().reset_index()
print('Era Profiles (National Averages):')
print(era_profiles.groupby('Era')[features].mean().round(2))

### 5.1 Determine Optimal Number of Clusters (Elbow Method)


In [ ]:
# Elbow Method and Silhouette Score Analysis
clustering_features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Poultry_per_HH']

df_2021_full = df_long[df_long['Year'] == 2021].copy()
X = df_2021_full[clustering_features].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calculate metrics for different k values
k_range = range(2, 11)
inertias = []
silhouette_scores = []

print("Evaluating K-Means for k = 2 to 10...")
print("=" * 60)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    sil_score = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(sil_score)
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette Score={sil_score:.3f}")


In [ ]:
# Plot Elbow Curve and Silhouette Scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow Curve
axes[0].plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Inertia (Within-cluster sum of squares)', fontsize=12)
axes[0].set_title('Elbow Method for Optimal k', fontsize=14, fontweight='bold')
axes[0].set_xticks(list(k_range))
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=4, color='r', linestyle='--', alpha=0.7, label='Suggested k=4')
axes[0].legend()

# Silhouette Score
axes[1].plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score for Different k', fontsize=14, fontweight='bold')
axes[1].set_xticks(list(k_range))
axes[1].grid(True, alpha=0.3)

best_k_sil = list(k_range)[np.argmax(silhouette_scores)]
axes[1].axvline(x=best_k_sil, color='r', linestyle='--', alpha=0.7, label=f'Best k={best_k_sil}')
axes[1].legend()

plt.tight_layout()
plt.savefig('temporal_elbow_silhouette.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nBest k based on Silhouette Score: {best_k_sil}")


### 5.2 Fit K-Means with Optimal k and Characterize Clusters


In [ ]:
# Cluster 2021 data
OPTIMAL_K = 4
clustering_features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Poultry_per_HH']

df_2021_full = df_long[df_long['Year'] == 2021].copy()
X = df_2021_full[clustering_features].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=OPTIMAL_K, random_state=42, n_init=10)
df_2021_full['Cluster'] = kmeans.fit_predict(X_scaled)

profile_names = {0: 'Commercial Hubs', 1: 'Subsistence Mixed', 2: 'Highland Pastoral', 3: 'Smallholder Diversified'}
df_2021_full['Profile'] = df_2021_full['Cluster'].map(profile_names)

print('2021 Cluster Distribution:')
print(df_2021_full['Cluster'].value_counts().sort_index())
print(f'Silhouette Score: {silhouette_score(X_scaled, df_2021_full["Cluster"]):.3f}')

In [ ]:
# Characterize clusters with detailed profiles
cluster_profiles = df_2021_full.groupby('Cluster')[clustering_features].mean()

print("Cluster Profiles (Mean Values):")
print("=" * 80)
display(cluster_profiles.round(2))

# Cluster distribution
print("\nCluster Distribution:")
print(df_2021_full['Cluster'].value_counts().sort_index())

# Detailed cluster analysis
print("\n" + "=" * 80)
print("DETAILED CLUSTER ANALYSIS")
print("=" * 80)

for cluster_id in range(OPTIMAL_K):
    cluster_data = df_2021_full[df_2021_full['Cluster'] == cluster_id]
    print(f"\n{'─' * 40}")
    print(f"CLUSTER {cluster_id}: {profile_names[cluster_id]}")
    print(f"{'─' * 40}")
    print(f"Number of Districts: {len(cluster_data)}")
    print(f"\nDistricts: {', '.join(cluster_data['Districts'].str.strip().tolist()[:10])}")
    if len(cluster_data) > 10:
        print(f"  ... and {len(cluster_data) - 10} more")
    print(f"\nKey Characteristics:")
    for feat in clustering_features:
        print(f"  - {feat}: {cluster_data[feat].mean():.2f}")


In [ ]:
# Cluster Visualization: Box plots for features across clusters
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, feature in enumerate(clustering_features):
    sns.boxplot(x='Cluster', y=feature, data=df_2021_full, ax=axes[idx], palette='husl')
    axes[idx].set_title(feature.replace('_', ' ').title(), fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Cluster')
    axes[idx].set_ylabel('')

plt.suptitle('Feature Distribution Across Clusters (2021)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('temporal_cluster_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Cluster Heatmap - Normalized profiles
cluster_means = df_2021_full.groupby('Cluster')[clustering_features].mean()
cluster_means_normalized = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min())

plt.figure(figsize=(14, 8))
sns.heatmap(cluster_means_normalized.T, annot=cluster_means.T.round(2), 
            cmap='YlOrRd', fmt='.2f', linewidths=0.5,
            xticklabels=[f'C{i}: {profile_names[i][:15]}' for i in range(OPTIMAL_K)],
            yticklabels=[f.replace('_', ' ').title() for f in clustering_features])
plt.title('Cluster Profiles: Feature Comparison Heatmap', fontsize=14, fontweight='bold')
plt.xlabel('Cluster (Profile)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.savefig('temporal_cluster_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()


## 6. Structural Change Detection

In [ ]:
def calculate_structural_change(district_data):
    first_decade = district_data[district_data['Year'] <= 1982]
    last_decade = district_data[district_data['Year'] >= 2012]
    if len(first_decade) == 0 or len(last_decade) == 0:
        return 0
    changes = []
    for col in clustering_features:
        early_mean = first_decade[col].mean()
        late_mean = last_decade[col].mean()
        if early_mean > 0:
            changes.append(abs((late_mean - early_mean) / early_mean))
    return np.mean(changes) * 100 if changes else 0

structural_changes = []
for district in df_long['Districts'].unique():
    district_data = df_long[df_long['Districts'] == district]
    change_index = calculate_structural_change(district_data)
    structural_changes.append({'District': district, 'Structural_Change_Index': change_index})

df_structural = pd.DataFrame(structural_changes).sort_values('Structural_Change_Index', ascending=False)
print('Top 15 Districts by Structural Change:')
print(df_structural.head(15).to_string(index=False))

In [ ]:
# Visualize structural change
fig, ax = plt.subplots(figsize=(12, 8))
top_20 = df_structural.head(20)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(top_20)))
ax.barh(top_20['District'], top_20['Structural_Change_Index'], color=colors)
ax.set_xlabel('Structural Change Index (%)')
ax.set_title('Top 20 Districts by Structural Change (1972-2021)', fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('structural_change_ranking.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Forecasting (2022-2030)

In [ ]:
forecast_years = list(range(2022, 2031))

forecast_df = pd.DataFrame({'Year': forecast_years})
for feature in features:
    model = LinearRegression()
    model.fit(national_trends['Year'].values.reshape(-1, 1), national_trends[feature].values)
    preds = model.predict(np.array(forecast_years).reshape(-1, 1))
    forecast_df[feature] = np.clip(preds, 0, None)

print('National Forecast (2022-2030):')
print(forecast_df.round(2))
forecast_df.to_csv('nepal_agriculture_forecast_2030.csv', index=False)

In [ ]:
# Visualize forecast
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()
plot_features = ['Poultry_per_HH', 'Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Goat_per_HH', 'Pig_per_HH']

for idx, feature in enumerate(plot_features):
    ax = axes[idx]
    ax.plot(national_trends['Year'], national_trends[feature], 'b-', linewidth=2, label='Historical')
    ax.plot(forecast_df['Year'], forecast_df[feature], 'r--', linewidth=2, label='Forecast')
    ax.fill_between(forecast_df['Year'], forecast_df[feature]*0.9, forecast_df[feature]*1.1, alpha=0.2, color='red')
    ax.set_xlabel('Year')
    ax.set_title(feature.replace('_', ' ').title(), fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axvline(x=2021, color='gray', linestyle=':', alpha=0.7)

plt.suptitle('Agricultural Trends: Historical + Forecast', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('trends_with_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Decision Tree Classification

In [ ]:
# Prepare data for classification
X_clf = df_2021_full[clustering_features].copy()
y_clf = df_2021_full['Cluster'].copy()

print("Classification Dataset:")
print(f"  Features: {X_clf.shape}")
print(f"  Target distribution:")
print(y_clf.value_counts().sort_index())

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X_clf, y_clf, 
    test_size=0.2, 
    random_state=42,
    stratify=y_clf  # Maintain cluster proportions
)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"\nTraining set distribution:")
print(y_train.value_counts().sort_index())
print(f"\nTesting set distribution:")
print(y_test.value_counts().sort_index())

# Train Decision Tree Classifier
dt = DecisionTreeClassifier(
    max_depth=4,              # Limit depth for interpretability
    min_samples_split=5,      # Minimum samples to split a node
    min_samples_leaf=2,       # Minimum samples in a leaf
    random_state=42
)

# Fit the model
dt.fit(X_train, y_train)

print("\n✓ Decision Tree Classifier trained successfully!")
print(f"\nTree depth: {dt.get_depth()}")
print(f"Number of leaves: {dt.get_n_leaves()}")

In [ ]:
# Model Evaluation
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Model Performance:")
print("=" * 50)
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Testing Accuracy:  {test_accuracy:.2%}")

# Detailed Classification Report
print("\nClassification Report (Test Set):")
print("=" * 60)
target_names = [f"Cluster {i}: {profile_names[i][:20]}" for i in range(OPTIMAL_K)]
print(classification_report(y_test, y_test_pred, target_names=target_names))


In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f'C{i}' for i in range(OPTIMAL_K)],
            yticklabels=[f'C{i}' for i in range(OPTIMAL_K)])
plt.xlabel('Predicted Cluster', fontsize=12)
plt.ylabel('Actual Cluster', fontsize=12)
plt.title('Confusion Matrix - Decision Tree Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('temporal_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Visualize the Decision Tree
plt.figure(figsize=(24, 12))
plot_tree(
    dt, 
    feature_names=clustering_features,
    class_names=[profile_names[i] for i in range(OPTIMAL_K)],
    filled=True,
    rounded=True,
    fontsize=10,
    proportion=True
)
plt.title('Decision Tree for Livestock Farming Profile Classification', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('temporal_decision_tree.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Extract and display decision rules in text format
print("Decision Tree Rules:")
print("=" * 80)
tree_rules = export_text(
    dt, 
    feature_names=clustering_features
)
print(tree_rules)


In [ ]:
# Feature Importance from Decision Tree
feature_importance = pd.DataFrame({
    'feature': clustering_features,
    'importance': dt.feature_importances_
}).sort_values('importance', ascending=True)

# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
plt.xlabel('Feature Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance in Decision Tree Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('temporal_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFeature Importance Ranking:")
print(feature_importance.sort_values('importance', ascending=False).to_string(index=False))

## 9. Policy Recommendations and Vulnerability Assessment


In [ ]:
# Generate policy recommendations based on cluster characteristics
print("\n" + "=" * 80)
print("POLICY RECOMMENDATIONS BY FARMING PROFILE")
print("=" * 80)

recommendations = {
    0: {
        'vulnerability': 'LOW-MEDIUM',
        'strengths': [
            'Good irrigation infrastructure',
            'Diversified livestock portfolio',
            'Access to markets'
        ],
        'challenges': [
            'Market price volatility',
            'Disease outbreak risks in dense populations'
        ],
        'recommendations': [
            'Establish livestock insurance schemes',
            'Develop cold storage and processing facilities',
            'Implement disease surveillance systems',
            'Promote cooperative marketing'
        ]
    },
    1: {
        'vulnerability': 'HIGH',
        'strengths': [
            'Traditional farming knowledge',
            'Low input dependency'
        ],
        'challenges': [
            'Limited irrigation',
            'Small land holdings',
            'Climate vulnerability',
            'Limited market access'
        ],
        'recommendations': [
            'Implement small-scale irrigation projects',
            'Provide goat/sheep farming subsidies',
            'Establish community pasture management',
            'Create mobile veterinary services',
            'Develop micro-credit programs'
        ]
    },
    2: {
        'vulnerability': 'MEDIUM-HIGH',
        'strengths': [
            'Large cattle holdings',
            'Highland adapted breeds',
            'Pastoral traditions'
        ],
        'challenges': [
            'Harsh climate conditions',
            'Limited infrastructure',
            'Seasonal migration patterns'
        ],
        'recommendations': [
            'Support traditional transhumance practices',
            'Improve mountain road connectivity',
            'Establish highland breed conservation programs',
            'Create seasonal veterinary camps',
            'Develop high-altitude fodder cultivation'
        ]
    },
    3: {
        'vulnerability': 'MEDIUM',
        'strengths': [
            'Diversified farming systems',
            'Moderate irrigation access',
            'Mixed livestock-crop integration'
        ],
        'challenges': [
            'Land fragmentation',
            'Labor migration',
            'Limited mechanization'
        ],
        'recommendations': [
            'Promote integrated farming systems',
            'Support small-scale dairy cooperatives',
            'Provide training in improved animal husbandry',
            'Develop local feed production',
            'Create farmer producer organizations'
        ]
    }
}

for cluster_id in range(OPTIMAL_K):
    rec = recommendations[cluster_id]
    print(f"\n{'─' * 60}")
    print(f"CLUSTER {cluster_id}: {profile_names[cluster_id]}")
    print(f"{'─' * 60}")
    print(f"\nVulnerability Level: {rec['vulnerability']}")
    
    print(f"\nStrengths:")
    for s in rec['strengths']:
        print(f"  ✓ {s}")
    
    print(f"\nChallenges:")
    for c in rec['challenges']:
        print(f"  ✗ {c}")
    
    print(f"\nPolicy Recommendations:")
    for i, r in enumerate(rec['recommendations'], 1):
        print(f"  {i}. {r}")


## 10. Conclusion and Summary

In [ ]:
# Comprehensive Summary
print("""
================================================================================
                              CONCLUSION
================================================================================

This study successfully applied machine learning techniques to analyze 50 years
of livestock farming data across Nepal's districts (1972-2021).

KEY ACHIEVEMENTS:

1. DATA GENERATION & EXPLORATION:
   - Generated synthetic historical data for 50 years based on 2021 baseline
   - Comprehensive EDA performed on all three source datasets
   - Feature engineering with 7 key derived indicators

2. TEMPORAL TREND ANALYSIS:
   - Identified major trends: Poultry commercialization, land fragmentation
   - Era-based analysis: Traditional (1972-1985), Early Modern (1986-2000), 
     Commercial (2001-2021)
   - Structural change detection across all districts

3. CLUSTERING ANALYSIS:
""")

print(f"   - Identified {OPTIMAL_K} distinct farming profiles using K-Means")
sil = silhouette_score(X_scaled, df_2021_full['Cluster'])
print(f"   - Achieved silhouette score of {sil:.3f}")
print("   - Profiles: Commercial Hubs, Subsistence Mixed, Highland Pastoral, Smallholder Diversified")

print(f"""
4. CLASSIFICATION MODEL:
   - Decision Tree classifier achieved {test_accuracy:.1%} test accuracy
   - Model provides interpretable rules for profile classification
   - Key factors identified through feature importance analysis

5. FORECASTING:
   - Linear trend projection to 2030
   - Identified continued poultry growth and land pressure trends

6. POLICY IMPLICATIONS:
   - Vulnerability assessment for each farming profile
   - Targeted recommendations for agricultural interventions
   - Evidence-based framework for resource allocation

================================================================================
""")

print("=" * 70)
print("FILES GENERATED:")
print("=" * 70)
output_files = [
    ('nepal_agriculture_50_years.csv', '50 years of synthetic district-level data'),
    ('nepal_agriculture_forecast_2030.csv', 'National forecast to 2030'),
    ('temporal_feature_correlation.png', 'Feature correlation heatmap'),
    ('temporal_feature_distributions.png', 'Feature distribution plots'),
    ('temporal_pairplot.png', 'Pairwise feature relationships'),
    ('temporal_elbow_silhouette.png', 'Cluster selection analysis'),
    ('temporal_cluster_boxplots.png', 'Feature distributions by cluster'),
    ('temporal_cluster_heatmap.png', 'Cluster profile heatmap'),
    ('national_trends_50years.png', '50-year trend analysis'),
    ('structural_change_ranking.png', 'District structural change ranking'),
    ('trends_with_forecast.png', 'Historical trends with forecast'),
    ('temporal_confusion_matrix.png', 'Classification confusion matrix'),
    ('temporal_decision_tree.png', 'Decision tree visualization'),
    ('temporal_feature_importance.png', 'Feature importance ranking')
]

for filename, description in output_files:
    print(f"  • {filename}: {description}")

print(f"\nTotal data points analyzed: {len(df_long)}")
print(f"Years covered: 1972-2021 (50 years)")
print(f"Districts: {df_long['Districts'].nunique()}")
print(f"Features: {len(features)}")

print("\n✓ 50-Year Temporal Analysis Complete!")